In [1]:
import KitNET as kit
import numpy as np
import pandas as pd
import time
import csv
from sklearn.metrics import f1_score,precision_score,recall_score

imports..

In [40]:
import pickle
with open("../X_train.pkl" , "rb") as f:
    X_train = pickle.load(f)
with open("../y_train.pkl" , "rb") as f:
    y_train = pickle.load(f)
with open("../X_test.pkl" , "rb") as f:
    X_test = pickle.load(f)
with open("../y_test.pkl" , "rb") as f:
    y_test = pickle.load(f)

Kitsune gets trained on a predefined number of samples which he takes as benign, then executes on the rest, so i have to make an input dataset

get only benign from the training set

In [41]:
train_df = X_train.copy()

train_df['label'] = y_train
train_one_class_df = train_df[train_df['label'] == 1 ]


get the testing set

In [42]:
test_df = X_test.copy()

test_df['label'] = y_test


sort them

In [43]:
train_one_class_df_sort = train_one_class_df.sort_index()
test_df_sort = test_df.sort_index()
y_test_sort = test_df_sort['label']

and concat both. dataset consists of 35000 benign only training and 50000 mixed testing 

In [46]:
y_test_sort.value_counts()

label
 1    35500
-1    14500
Name: count, dtype: int64

testing comes first with benign and then only malicious treating dataset like the original kitsune example, thats why the sort

In [44]:
Kitsune_df = pd.concat([train_one_class_df_sort,test_df_sort])

In [39]:
X_kitsune = Kitsune_df.drop('label',axis=1)
X_kitsune

,0,1,2,3,4,5,6,7,8,9,...,105,106,107,108,109,110,111,112,113,114
6,1.99938,114.0000,1.085560e-04,1.99972,113.9980,1.116200e-01,2.09528,111.9170,106.230000,5.11994,...,0.000000e+00,0.000000e+00,0.0,1.00000,114.0000,0.000000e+00,114.0000,0.000000e+00,0.000000e+00,0.000000e+00
7,1.86453,102.8030,1.825060e+02,2.48821,103.0490,1.628830e+02,7.14736,102.3650,159.065000,78.60640,...,0.000000e+00,0.000000e+00,0.0,1.00000,114.0000,0.000000e+00,161.2200,0.000000e+00,0.000000e+00,0.000000e+00
8,2.85173,106.7290,1.470560e+02,3.47795,106.1980,1.406160e+02,8.13752,103.7950,154.109000,79.59550,...,0.000000e+00,0.000000e+00,0.0,1.00000,114.0000,0.000000e+00,161.2200,0.000000e+00,0.000000e+00,0.000000e+00
12,2.89126,90.5670,5.902230e+02,3.71838,93.7737,5.224110e+02,8.49585,98.6401,335.154000,79.94450,...,9.094950e-13,-6.251840e-36,0.0,7.95863,60.0000,9.536740e-07,84.8528,1.286220e-12,2.484300e-29,2.731520e-17
20,1.32915,75.1596,3.638940e+01,1.63052,77.7364,8.525630e+01,3.16704,85.4821,220.716000,27.68580,...,2.254730e+02,0.000000e+00,0.0,2.93182,82.8717,1.551040e+01,133.7640,2.405720e+02,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99989,7.91606,82.2990,2.302610e+00,10.74610,82.3585,2.739510e+00,22.64480,82.2661,2.227450,74.76050,...,2.728480e-12,0.000000e+00,0.0,58.22340,82.0000,1.348700e-06,82.0000,1.818990e-12,0.000000e+00,0.000000e+00
99993,1.86017,88.0000,1.818990e-12,1.91359,88.0000,9.094950e-13,1.97052,88.0000,0.000024,6.68654,...,0.000000e+00,0.000000e+00,0.0,1.99970,88.0000,9.536740e-07,88.0000,9.094950e-13,0.000000e+00,0.000000e+00
99994,1.00000,110.0000,2.218020e-07,1.00000,110.0000,1.293440e-03,1.01315,109.6890,7.377100,2.54652,...,1.384710e+02,0.000000e+00,0.0,21.49450,90.1841,9.105580e+00,90.1841,8.291160e+01,0.000000e+00,0.000000e+00
99996,2.00582,60.0655,1.435830e+00,2.11972,61.1742,2.445450e+01,6.61853,71.7619,120.420000,108.64700,...,0.000000e+00,0.000000e+00,0.0,1.00000,60.0000,0.000000e+00,60.0000,0.000000e+00,0.000000e+00,0.000000e+00


In [45]:
# KitNET params:
maxAE = 10 #maximum size for any autoencoder in the ensemble layer
FMgrace = 5000 #the number of instances taken to learn the feature mapping (the ensemble's architecture)
ADgrace = 30500 #the number of instances used to train the anomaly detector (ensemble itself)

# Build KitNET
K = kit.KitNET(X_kitsune.shape[1],maxAE,FMgrace,ADgrace)
RMSEs = np.zeros(X_kitsune.shape[0]) # a place to save the scores

print("Running KitNET:")
start = time.time()
# Here we process (train/execute) each individual observation.
# In this way, X is essentially a stream, and each observation is discarded after performing process() method.
for i in range(X_kitsune.shape[0]):
    if i % 1000 == 0:
        print(i)
    RMSEs[i] = K.process(X_kitsune.iloc[i]) #will train during the grace periods, then execute on all the rest.
stop = time.time()
print("Complete. Time elapsed: "+ str(stop - start))

Feature-Mapper: train-mode, Anomaly-Detector: off-mode
Running KitNET:
0
1000
2000
3000
4000
5000
The Feature-Mapper found a mapping: 115 features to 19 autoencoders.
Feature-Mapper: execute-mode, Anomaly-Detector: train-mode
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
Feature-Mapper: execute-mode, Anomaly-Detector: exeute-mode
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
Complete. Time elapsed: 1019.2790184020996


In [7]:
# Here we demonstrate how one can fit the RMSE scores to a log-normal distribution (useful for finding/setting a cutoff threshold \phi)
from scipy.stats import norm
benignSample = np.log(RMSEs[FMgrace+ADgrace+1:71000])
logProbs = norm.logsf(np.log(RMSEs), np.mean(benignSample), np.std(benignSample))

In [47]:
# Calculate and plot F1 scores
thresholds = np.linspace(min(logProbs), max(logProbs), num=50)
f1_scores = []
precision_scores = []
recall_scores = []

labels = np.zeros(85500, dtype=int)

# Set the last part of the array to ones
labels[71000:] = 1


best_score = 0
for threshold in thresholds:
     
    predictions = (logProbs < threshold).astype(int)
    f11_score = f1_score(y_true = labels, y_pred = predictions,average = 'weighted')
    f1_scores.append(f11_score)
    precision_scores.append(precision_score(y_true = labels, y_pred = predictions,pos_label = 1,average = 'weighted'))
    recall_scores.append(recall_score(y_true = labels, y_pred = predictions,pos_label = 1,average = 'weighted'))
    if f11_score > best_score:
        best_score = f11_score
        print("best score is "+ str(best_score))
        best_precision = precision_scores[-1]
        best_recall = recall_scores[-1]
        best_threshold = threshold
    

print("best f1 score = "+ str(best_score))
print("precision score = "+ str(best_precision))
print("recall score = "+ str(best_recall))
print("best threshold score = "+ str(best_threshold))

/home/goikon/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


best score is 0.7534704706387907
best score is 0.792198308936773
best score is 0.8349139795252835
best score is 0.858718118964745
best score is 0.8863425822988168
best score is 0.9057551723975802
best score is 0.9193503158469769
best score is 0.9302881588136841
best score is 0.9366612281389098
best score is 0.9416276170348051
best score is 0.9457839033528682
best score is 0.948435090373905
best score is 0.9500207358351997
best score is 0.9511180722650054
best score is 0.9523581552874748
best score is 0.9531822742468207
best score is 0.9540705372166349
best score is 0.9547186509743365
best score is 0.9555896192009193
best score is 0.9568918066198911
best score is 0.9583588964328197
best score is 0.9596634106915153
best score is 0.9609369734357348
best score is 0.9616366998210553
best score is 0.9623091580996906
best score is 0.962631985895245
best score is 0.96303187108805
best score is 0.9635085454514041
best score is 0.9642158996146464
best score is 0.9645112734560066
best score is 0.

Okaay, here's the score for the Kitsune 0.9655 F1 Score Impressive for outlier detection, better than OCSVM (0.9377) 